In [2]:
import sys
sys.path.insert(0, "..")

In [9]:
import graph
import importlib
importlib.reload(graph)
import uuid
v1 = graph.Vertex(str(uuid.uuid4()), {}, 1000, 10, 10, int(1e8))
v21 = graph.Vertex(str(uuid.uuid4()), {}, 1000, 10, 10, int(1e8))
v22 = graph.Vertex(str(uuid.uuid4()), {}, 1000, 10, 10, int(1e8))
v3 = graph.Vertex(str(uuid.uuid4()), {}, 0, 0, 10, int(1e8))
g = graph.ExecutionGraph(str(uuid.uuid4()))
g.add_vertex(v1)
g.add_vertex(v21)
g.add_vertex(v22)
g.add_vertex(v3)
g.connect(v1, v21, 1000)
g.connect(v1, v22, 100)
g.connect(v21, v3, 50)
g.connect(v22, v3, 500)

In [10]:
print(g.get_vertexs())
print(g.get_edges())
print(g.get_sources())
print(g.get_sinks())
print(g.get_edge(*g.get_edges()[0]))
v_from, v_to = g.get_edges()[0]
print(g.g.edges[v_from, v_to])

[Vertex(uuid='817af302-ae30-43af-ac28-0480ded8cd73', domain_constraint={}, out_unit_size=1000, out_unit_rate=10, mi=10, memory=100000000), Vertex(uuid='3cff1e49-d64a-499a-a21d-90da569167a8', domain_constraint={}, out_unit_size=1000, out_unit_rate=10, mi=10, memory=100000000), Vertex(uuid='441f6fa8-3020-4012-b8f7-068df1e826ce', domain_constraint={}, out_unit_size=1000, out_unit_rate=10, mi=10, memory=100000000), Vertex(uuid='cec63da0-6681-48cb-b0ae-baef6e3d5d59', domain_constraint={}, out_unit_size=0, out_unit_rate=0, mi=10, memory=100000000)]
[('817af302-ae30-43af-ac28-0480ded8cd73', '3cff1e49-d64a-499a-a21d-90da569167a8'), ('817af302-ae30-43af-ac28-0480ded8cd73', '441f6fa8-3020-4012-b8f7-068df1e826ce'), ('3cff1e49-d64a-499a-a21d-90da569167a8', 'cec63da0-6681-48cb-b0ae-baef6e3d5d59'), ('441f6fa8-3020-4012-b8f7-068df1e826ce', 'cec63da0-6681-48cb-b0ae-baef6e3d5d59')]
[Vertex(uuid='817af302-ae30-43af-ac28-0480ded8cd73', domain_constraint={}, out_unit_size=1000, out_unit_rate=10, mi=10, me

In [11]:
import min_cut
importlib.reload(min_cut)

<module 'min_cut' from 'd:\\Graduate2\\repo\\simple-sim\\algo\\min_cut.py'>

In [12]:
s_cut, t_cut = min_cut.min_cut(g)
print(s_cut, t_cut)

{'817af302-ae30-43af-ac28-0480ded8cd73', '3cff1e49-d64a-499a-a21d-90da569167a8'} {'441f6fa8-3020-4012-b8f7-068df1e826ce', 'cec63da0-6681-48cb-b0ae-baef6e3d5d59'}
